# Tutorial 3: Conectividad Cerebral con TDA

## Análisis Topológico de Redes Cerebrales

**Autor:** MARK-126  
**Nivel:** Avanzado  
**Tiempo estimado:** 150-180 minutos

---

## Objetivos de Aprendizaje

1. ✅ Analizar conectomas cerebrales con TDA
2. ✅ Comparar redes funcionales vs estructurales
3. ✅ Detectar comunidades usando topología
4. ✅ Procesar matrices de correlación fMRI
5. ✅ Identificar biomarcadores topológicos

---

## 1. Introducción a la Conectividad Cerebral

### 1.1 ¿Qué es un Conectoma?

Un **conectoma** es un mapa completo de las conexiones en el cerebro:

**Tipos de conectividad:**

1. **Conectividad Estructural:**
   - Conexiones físicas (axones, sinapsis)
   - Medida con: DTI (Diffusion Tensor Imaging)
   - Relativamente estable en el tiempo

2. **Conectividad Funcional:**
   - Correlación de actividad entre regiones
   - Medida con: fMRI, EEG, MEG
   - Dinámica, cambia con estados cognitivos

3. **Conectividad Efectiva:**
   - Influencia causal entre regiones
   - Requiere análisis temporal

### 1.2 ¿Por qué TDA para Conectomas?

TDA ofrece ventajas únicas:
- **Invarianza:** Robusto a ruido y elección de umbrales
- **Multi-escala:** Captura estructura en todas las escalas
- **Características globales:** Más allá de medidas locales (grado, clustering)
- **Biomarcadores:** Características topológicas para diagnóstico

---

In [ ]:
# Importaciones
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# TDA
from ripser import ripser
from persim import plot_diagrams, bottleneck, sliced_wasserstein
import gudhi as gd

# Análisis de redes
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Visualización
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Config
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.1)
np.random.seed(42)

print("✅ Bibliotecas importadas correctamente")

---

## 2. Construcción de Matrices de Conectividad

### 2.1 Matriz de Correlación Funcional

Dadas señales temporales de N regiones cerebrales:
$$C_{ij} = \text{corr}(X_i(t), X_j(t))$$

### 2.2 De Matriz a Complejo Simplicial

Dos enfoques:

1. **Filtración por umbral:**
   - Conecta regiones si $C_{ij} > \theta$
   - Varía $\theta$ para obtener filtración

2. **Filtración por distancia:**
   - Define distancia: $d_{ij} = 1 - |C_{ij}|$
   - Usa Vietoris-Rips en espacio de distancias

---

## 3. Generación de Datos Sintéticos de fMRI

In [ ]:
def generate_fmri_timeseries(n_regions=50, n_timepoints=200, 
                             n_communities=3, noise_level=0.3):
    """
    Genera series temporales sintéticas de fMRI con estructura de comunidades.
    
    Parameters:
    -----------
    n_regions : int
        Número de regiones cerebrales (ROIs)
    n_timepoints : int
        Número de puntos temporales
    n_communities : int
        Número de comunidades funcionales
    noise_level : float
        Nivel de ruido (0-1)
    """
    # Dividir regiones en comunidades
    regions_per_community = n_regions // n_communities
    
    timeseries = np.zeros((n_regions, n_timepoints))
    labels = np.zeros(n_regions, dtype=int)
    
    # Crear señales base para cada comunidad
    for comm in range(n_communities):
        start_idx = comm * regions_per_community
        end_idx = start_idx + regions_per_community if comm < n_communities - 1 else n_regions
        
        # Señal común de la comunidad (BOLD response simulada)
        t = np.linspace(0, 4*np.pi, n_timepoints)
        common_signal = np.sin(t + comm * np.pi/3) + 0.5 * np.sin(2*t + comm)
        
        # Agregar a cada región de la comunidad
        for i in range(start_idx, end_idx):
            # Señal común + ruido individual
            timeseries[i] = common_signal + np.random.randn(n_timepoints) * noise_level
            labels[i] = comm
    
    # Agregar correlaciones inter-comunidades (más débiles)
    global_signal = np.sin(t) * 0.2
    timeseries += global_signal
    
    return timeseries, labels


def compute_functional_connectivity(timeseries):
    """
    Calcula matriz de conectividad funcional (correlación).
    """
    n_regions = timeseries.shape[0]
    conn_matrix = np.zeros((n_regions, n_regions))
    
    for i in range(n_regions):
        for j in range(i, n_regions):
            corr, _ = pearsonr(timeseries[i], timeseries[j])
            conn_matrix[i, j] = corr
            conn_matrix[j, i] = corr
    
    return conn_matrix


# Generar datos sintéticos
print("🧠 Generando series temporales fMRI sintéticas...\n")
n_rois = 60
fmri_data, true_labels = generate_fmri_timeseries(
    n_regions=n_rois, 
    n_timepoints=250,
    n_communities=3,
    noise_level=0.4
)

# Calcular conectividad funcional
fc_matrix = compute_functional_connectivity(fmri_data)

print(f"✅ Datos generados:")
print(f"   • {n_rois} regiones cerebrales (ROIs)")
print(f"   • 250 puntos temporales")
print(f"   • 3 comunidades funcionales")
print(f"\n📊 Matriz de conectividad: {fc_matrix.shape}")
print(f"   • Rango de correlaciones: [{fc_matrix.min():.3f}, {fc_matrix.max():.3f}]")

### Visualización de Datos fMRI

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Series temporales de algunas ROIs
ax1 = axes[0, 0]
sample_rois = [0, 20, 40]  # Una de cada comunidad
colors = ['#e74c3c', '#3498db', '#2ecc71']
for idx, roi in enumerate(sample_rois):
    ax1.plot(fmri_data[roi], label=f'ROI {roi} (Comunidad {true_labels[roi]})',
            color=colors[idx], linewidth=2, alpha=0.7)
ax1.set_xlabel('Tiempo (TRs)', fontsize=11)
ax1.set_ylabel('Señal BOLD', fontsize=11)
ax1.set_title('Series Temporales fMRI (ejemplos)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Matriz de conectividad funcional
ax2 = axes[0, 1]
im = ax2.imshow(fc_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
ax2.set_xlabel('ROI', fontsize=11)
ax2.set_ylabel('ROI', fontsize=11)
ax2.set_title('Matriz de Conectividad Funcional', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax2, label='Correlación de Pearson')

# Marcar comunidades
for i in range(3):
    start = i * 20
    end = (i + 1) * 20 if i < 2 else n_rois
    ax2.axhline(start, color='white', linewidth=2)
    ax2.axvline(start, color='white', linewidth=2)

# 3. Distribución de correlaciones
ax3 = axes[1, 0]
# Solo triángulo superior (sin diagonal)
triu_indices = np.triu_indices_from(fc_matrix, k=1)
correlations = fc_matrix[triu_indices]
ax3.hist(correlations, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax3.axvline(0, color='red', linestyle='--', linewidth=2, label='Correlación = 0')
ax3.set_xlabel('Correlación', fontsize=11)
ax3.set_ylabel('Frecuencia', fontsize=11)
ax3.set_title('Distribución de Correlaciones Funcionales', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# 4. Grafo de red (top correlaciones)
ax4 = axes[1, 1]
threshold = np.percentile(correlations, 90)  # Top 10%
G = nx.Graph()
for i in range(n_rois):
    G.add_node(i)

for i in range(n_rois):
    for j in range(i+1, n_rois):
        if fc_matrix[i, j] > threshold:
            G.add_edge(i, j, weight=fc_matrix[i, j])

# Layout
pos = nx.spring_layout(G, seed=42, k=0.5)
node_colors = [colors[label] for label in true_labels]

nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                       node_size=100, alpha=0.8, ax=ax4)
nx.draw_networkx_edges(G, pos, alpha=0.2, ax=ax4)
ax4.set_title(f'Red Funcional (correlación > {threshold:.2f})', 
             fontsize=12, fontweight='bold')
ax4.axis('off')

plt.tight_layout()
plt.show()

print(f"\n📊 Red funcional:")
print(f"   • Nodos: {G.number_of_nodes()}")
print(f"   • Aristas: {G.number_of_edges()}")
print(f"   • Densidad: {nx.density(G):.3f}")

---

## 4. Análisis Topológico de Conectomas

### 4.1 Filtraciones de Conectividad

Vamos a crear una filtración variando el umbral de correlación.

---

In [ ]:
def connectivity_to_distance(conn_matrix):
    """
    Convierte matriz de conectividad (correlación) a matriz de distancia.
    """
    # Distancia = 1 - |correlación|
    # Usamos valor absoluto para considerar anti-correlaciones también
    dist_matrix = 1 - np.abs(conn_matrix)
    np.fill_diagonal(dist_matrix, 0)  # Diagonal = 0
    return dist_matrix


def analyze_connectivity_topology(conn_matrix, maxdim=2):
    """
    Analiza la topología de una matriz de conectividad.
    """
    # Convertir a distancias
    dist_matrix = connectivity_to_distance(conn_matrix)
    
    # Calcular persistencia usando matriz de distancia
    result = ripser(dist_matrix, maxdim=maxdim, distance_matrix=True, thresh=1.0)
    
    return result['dgms']


# Analizar topología del conectoma
print("⏳ Calculando homología persistente del conectoma...\n")
diagrams_fc = analyze_connectivity_topology(fc_matrix, maxdim=2)

print("✅ Análisis completado")
print(f"\n📊 Características topológicas:")
print(f"   • H₀ (componentes): {len(diagrams_fc[0])}")
print(f"   • H₁ (ciclos): {len(diagrams_fc[1])}")
print(f"   • H₂ (cavidades): {len(diagrams_fc[2])}")

# Visualizar diagrama de persistencia
fig, ax = plt.subplots(figsize=(10, 8))
plot_diagrams(diagrams_fc, ax=ax)
ax.set_title('Diagrama de Persistencia: Conectoma Funcional', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 💡 Interpretación Neurobiológica

**H₀ (Componentes conectadas):**
- Muchas componentes inicialmente → ROIs funcionalmente independientes
- Se fusionan gradualmente → integración funcional
- Una componente final → red globalmente conectada

**H₁ (Ciclos):**
- Representan **circuitos de retroalimentación**
- Fundamental para procesamiento recurrente
- Más ciclos = mayor complejidad funcional

**H₂ (Cavidades):**
- Estructuras de alta dimensión
- Indican organización jerárquica
- Relacionado con procesamiento multimodal

---

## 5. Comparación: Diferentes Estados Cerebrales

In [ ]:
# Generar diferentes estados
print("🧠 Generando diferentes estados cerebrales...\n")

states = {
    'Reposo': generate_fmri_timeseries(n_rois, 250, n_communities=2, noise_level=0.5)[0],
    'Tarea Cognitiva': generate_fmri_timeseries(n_rois, 250, n_communities=4, noise_level=0.3)[0],
    'Sueño': generate_fmri_timeseries(n_rois, 250, n_communities=1, noise_level=0.2)[0]
}

# Calcular conectividad y topología para cada estado
conn_matrices = {}
diagrams_states = {}

for state_name, timeseries in states.items():
    print(f"⏳ Procesando estado: {state_name}")
    conn = compute_functional_connectivity(timeseries)
    conn_matrices[state_name] = conn
    diagrams_states[state_name] = analyze_connectivity_topology(conn, maxdim=1)
    print(f"   H₁: {len(diagrams_states[state_name][1])} ciclos\n")

# Visualizar matrices de conectividad
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (state_name, conn) in enumerate(conn_matrices.items()):
    im = axes[idx].imshow(conn, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    axes[idx].set_title(f'{state_name}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('ROI')
    axes[idx].set_ylabel('ROI')
    plt.colorbar(im, ax=axes[idx])

plt.suptitle('Matrices de Conectividad Funcional por Estado', 
            fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Visualizar diagramas de persistencia
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (state_name, diagrams) in enumerate(diagrams_states.items()):
    plot_diagrams(diagrams, ax=axes[idx])
    axes[idx].set_title(f'{state_name}\nH₁ = {len(diagrams[1])} ciclos', 
                       fontsize=12, fontweight='bold')

plt.suptitle('Diagramas de Persistencia por Estado Cerebral', 
            fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Matriz de Distancias entre Estados

In [ ]:
# Calcular distancias topológicas entre estados
state_names = list(diagrams_states.keys())
n_states = len(state_names)

# Usar H₁ (ciclos) para comparación
distance_matrix = np.zeros((n_states, n_states))

for i, name1 in enumerate(state_names):
    for j, name2 in enumerate(state_names):
        if i <= j:
            dgm1 = diagrams_states[name1][1]
            dgm2 = diagrams_states[name2][1]
            
            if len(dgm1) > 0 and len(dgm2) > 0:
                dist = bottleneck(dgm1, dgm2)
                distance_matrix[i, j] = dist
                distance_matrix[j, i] = dist

# Visualizar
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(distance_matrix, cmap='YlOrRd', aspect='auto')
ax.set_xticks(range(n_states))
ax.set_yticks(range(n_states))
ax.set_xticklabels(state_names, rotation=45)
ax.set_yticklabels(state_names)
ax.set_title('Distancia Topológica entre Estados\n(Bottleneck en H₁)', 
            fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Distancia de Bottleneck')

# Agregar valores
for i in range(n_states):
    for j in range(n_states):
        text = ax.text(j, i, f'{distance_matrix[i, j]:.3f}',
                      ha="center", va="center", 
                      color="black", fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

print("\n🔍 Interpretación:")
print(f"   • Estados más similares: {state_names[0]} y {state_names[1]}")
print(f"     (distancia: {distance_matrix[0,1]:.3f})")
print(f"\n   • El estado de Sueño muestra estructura topológica diferente")
print(f"     debido a mayor sincronización (menos comunidades)")

---

## 6. Detección de Comunidades Topológicas

Usaremos características topológicas para identificar comunidades funcionales.

---

In [ ]:
def detect_communities_spectral(conn_matrix, n_clusters=3):
    """
    Detecta comunidades usando clustering espectral.
    """
    # Asegurar que la matriz sea positiva
    affinity = np.abs(conn_matrix)
    
    clustering = SpectralClustering(n_clusters=n_clusters, 
                                   affinity='precomputed',
                                   random_state=42)
    labels = clustering.fit_predict(affinity)
    
    return labels


# Detectar comunidades en el estado de reposo
print("🔍 Detectando comunidades funcionales...\n")
detected_labels = detect_communities_spectral(fc_matrix, n_clusters=3)

# Comparar con comunidades verdaderas
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

ari = adjusted_rand_score(true_labels, detected_labels)
nmi = normalized_mutual_info_score(true_labels, detected_labels)

print(f"✅ Detección completada")
print(f"\n📊 Calidad de detección:")
print(f"   • Adjusted Rand Index: {ari:.3f}")
print(f"   • Normalized Mutual Info: {nmi:.3f}")
print(f"\n   (valores cercanos a 1.0 = detección perfecta)")

# Visualizar comunidades detectadas vs verdaderas
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Ordenar matriz por comunidades verdaderas
true_order = np.argsort(true_labels)
fc_sorted_true = fc_matrix[true_order][:, true_order]

im1 = axes[0].imshow(fc_sorted_true, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
axes[0].set_title('Comunidades Verdaderas', fontsize=12, fontweight='bold')
axes[0].set_xlabel('ROI (ordenado)')
axes[0].set_ylabel('ROI (ordenado)')
plt.colorbar(im1, ax=axes[0])

# Ordenar por comunidades detectadas
detected_order = np.argsort(detected_labels)
fc_sorted_detected = fc_matrix[detected_order][:, detected_order]

im2 = axes[1].imshow(fc_sorted_detected, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
axes[1].set_title(f'Comunidades Detectadas (ARI={ari:.2f})', 
                 fontsize=12, fontweight='bold')
axes[1].set_xlabel('ROI (ordenado)')
axes[1].set_ylabel('ROI (ordenado)')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

---

## 7. Biomarcadores Topológicos

Características topológicas como biomarcadores de estados cerebrales.

---

In [ ]:
def extract_connectivity_features(conn_matrix):
    """
    Extrae características topológicas y de grafo de una matriz de conectividad.
    """
    features = {}
    
    # 1. Características de grafo tradicionales
    G = nx.from_numpy_array(np.abs(conn_matrix))
    features['avg_clustering'] = nx.average_clustering(G, weight='weight')
    features['avg_degree'] = np.mean([d for n, d in G.degree(weight='weight')])
    features['density'] = nx.density(G)
    
    # 2. Características topológicas
    diagrams = analyze_connectivity_topology(conn_matrix, maxdim=2)
    
    # H₁ (ciclos)
    if len(diagrams[1]) > 0:
        dgm1 = diagrams[1][np.isfinite(diagrams[1][:, 1])]
        if len(dgm1) > 0:
            lifetimes_1 = dgm1[:, 1] - dgm1[:, 0]
            features['n_cycles'] = len(dgm1)
            features['max_cycle_persistence'] = np.max(lifetimes_1)
            features['mean_cycle_persistence'] = np.mean(lifetimes_1)
        else:
            features['n_cycles'] = 0
            features['max_cycle_persistence'] = 0
            features['mean_cycle_persistence'] = 0
    else:
        features['n_cycles'] = 0
        features['max_cycle_persistence'] = 0
        features['mean_cycle_persistence'] = 0
    
    # H₂ (cavidades)
    if len(diagrams[2]) > 0:
        dgm2 = diagrams[2][np.isfinite(diagrams[2][:, 1])]
        features['n_cavities'] = len(dgm2)
    else:
        features['n_cavities'] = 0
    
    return features


# Extraer características para cada estado
print("📊 Extrayendo biomarcadores topológicos...\n")
biomarkers = {}

for state_name, conn in conn_matrices.items():
    print(f"⏳ Procesando: {state_name}")
    biomarkers[state_name] = extract_connectivity_features(conn)

# Crear DataFrame
df_biomarkers = pd.DataFrame(biomarkers).T
print("\n✅ Características extraídas:\n")
print(df_biomarkers.to_string())

# Visualizar biomarcadores
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

features_to_plot = list(df_biomarkers.columns)
colors_bar = ['#e74c3c', '#3498db', '#2ecc71']

for idx, feature in enumerate(features_to_plot):
    values = df_biomarkers[feature].values
    bars = axes[idx].bar(state_names, values, color=colors_bar, 
                        alpha=0.7, edgecolor='black', linewidth=2)
    axes[idx].set_ylabel(feature, fontsize=10)
    axes[idx].set_title(f'{feature}', fontsize=11, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, axis='y')
    axes[idx].tick_params(axis='x', rotation=45)
    
    # Valores en barras
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.2f}',
                      ha='center', va='bottom', fontweight='bold')

plt.suptitle('Biomarcadores Topológicos por Estado Cerebral', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 🩺 Interpretación Clínica

Estos biomarcadores topológicos pueden usarse para:

1. **Clasificación de estados:** Distinguir reposo vs tarea vs sueño
2. **Diagnóstico:** Detectar patrones anormales en trastornos neurológicos
3. **Seguimiento:** Monitorear cambios durante tratamiento
4. **Predicción:** Anticipar transiciones entre estados

**Ejemplos clínicos:**
- **Alzheimer:** Reducción en número de ciclos (H₁)
- **Esquizofrenia:** Alteración en estructura de cavidades (H₂)
- **Epilepsia:** Cambios súbitos en persistencia antes de crisis

---

## 8. Ejercicios Prácticos

### Ejercicio 1: Analiza tu propia matriz

Genera una matriz de conectividad con parámetros diferentes y analízala:

```python
# Tu código aquí
```

---

In [ ]:
# Espacio para Ejercicio 1


### Ejercicio 2: Clasificador de estados

Usa biomarcadores topológicos para entrenar un clasificador:

```python
from sklearn.ensemble import RandomForestClassifier

# 1. Genera múltiples ejemplos de cada estado
# 2. Extrae biomarcadores
# 3. Entrena clasificador
# 4. Evalúa precisión
```

---

In [ ]:
# Espacio para Ejercicio 2


## 9. Resumen

### ✅ Lo que aprendimos:

1. **Conectomas:** Tipos y representaciones
2. **Matrices de conectividad:** De correlación a topología
3. **Análisis topológico:** Aplicado a redes cerebrales
4. **Comparación de estados:** Distancias topológicas
5. **Detección de comunidades:** Clustering funcional
6. **Biomarcadores:** Características para diagnóstico

### 🔑 Puntos Clave:

- TDA captura estructura multi-escala de conectomas
- H₁ (ciclos) representa retroalimentación funcional
- H₂ (cavidades) indica organización jerárquica
- Biomarcadores topológicos son robustos y clínicamente relevantes

---

## Referencias

1. Petri et al. (2014). "Homological scaffolds of brain functional networks". *J. Royal Society Interface*
2. Sizemore et al. (2018). "Cliques and cavities in the human connectome". *J. Comp. Neuroscience*
3. Lord et al. (2016). "Insights into brain architectures from the homological scaffolds". *Frontiers*

---

**Próximo:** Tutorial 4 - Algoritmo Mapper para visualización

**Autor:** MARK-126  
**Licencia:** MIT